In [53]:
import openml
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder #For encoding categorical variables
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans,AgglomerativeClustering,DBSCAN,AffinityPropagation,MeanShift,SpectralClustering
from sklearn.metrics import accuracy_score,f1_score,adjusted_rand_score,silhouette_score
from joblib import Parallel,delayed
import time
import scipy

In [54]:

pip install pyamg

  Using cached pyamg-4.1.0.tar.gz (749 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pyamg
Failed to build pyamg
    Running setup.py install for pyamg: started
    Running setup.py install for pyamg: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Moditha\AppData\Local\Programs\Python\Python39\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Moditha\\AppData\\Local\\Temp\\pip-install-kvjaghvj\\pyamg_a9c9b3b489954ac39b47dfa8c93e32a0\\setup.py'"'"'; __file__='"'"'C:\\Users\\Moditha\\AppData\\Local\\Temp\\pip-install-kvjaghvj\\pyamg_a9c9b3b489954ac39b47dfa8c93e32a0\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Moditha\AppData\Local\Temp\pip-wheel-ated_2ef'
       cwd: C:\Users\Moditha\AppData\Local\Temp\pip-install-kvjaghvj\pyamg_a9c9b3b489954ac39b47dfa8c93e32a0\
  Complete output (141 lines):
  C:\Users\Moditha\AppData\Local\Temp\pip-install-kvjaghvj\pyamg_a9c9b3b489954ac

    copying pyamg\version.py -> build\lib.win-amd64-3.9\pyamg
    copying pyamg\__init__.py -> build\lib.win-amd64-3.9\pyamg
    creating build\lib.win-amd64-3.9\pyamg\aggregation
    copying pyamg\aggregation\adaptive.py -> build\lib.win-amd64-3.9\pyamg\aggregation
    copying pyamg\aggregation\aggregate.py -> build\lib.win-amd64-3.9\pyamg\aggregation
    copying pyamg\aggregation\aggregation.py -> build\lib.win-amd64-3.9\pyamg\aggregation
    copying pyamg\aggregation\rootnode.py -> build\lib.win-amd64-3.9\pyamg\aggregation
    copying pyamg\aggregation\smooth.py -> build\lib.win-amd64-3.9\pyamg\aggregation
    copying pyamg\aggregation\tentative.py -> build\lib.win-amd64-3.9\pyamg\aggregation
    copying pyamg\aggregation\__init__.py -> build\lib.win-amd64-3.9\pyamg\aggregation
    creating build\lib.win-amd64-3.9\pyamg\amg_core
    copying pyamg\amg_core\bindthem.py -> build\lib.win-amd64-3.9\pyamg\amg_core
    copying pyamg\amg_core\__init__.py -> build\lib.win-amd64-3.9\pyamg\amg

# Download Datasets Using openml

In [6]:
iris = openml.datasets.get_dataset("iris")
iris_df, iris_label, categorical_indicator, attribute_names = iris.get_data(
    target=iris.default_target_attribute, dataset_format="dataframe"
)
iris_df["class"]=iris_label
iris_x=iris_df.iloc[:,:4]
iris_df

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [7]:
wine = openml.datasets.get_dataset("wine")
wine_df, wine_label, categorical_indicator, attribute_names = wine.get_data(
    target= wine.default_target_attribute, dataset_format="dataframe"
)
wine_df["class"]=wine_label
wine_x=wine_df.iloc[:,:13]
wine_df

,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280%2FOD315_of_diluted_wines,Proline,class
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,3
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,3
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,3
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,3


# Identify Data Types

In [8]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   sepallength  150 non-null    float64 
 1   sepalwidth   150 non-null    float64 
 2   petallength  150 non-null    float64 
 3   petalwidth   150 non-null    float64 
 4   class        150 non-null    category
dtypes: category(1), float64(4)
memory usage: 5.1 KB


In [9]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   Alcohol                         178 non-null    float64 
 1   Malic_acid                      178 non-null    float64 
 2   Ash                             178 non-null    float64 
 3   Alcalinity_of_ash               178 non-null    float64 
 4   Magnesium                       178 non-null    uint8   
 5   Total_phenols                   178 non-null    float64 
 6   Flavanoids                      178 non-null    float64 
 7   Nonflavanoid_phenols            178 non-null    float64 
 8   Proanthocyanins                 178 non-null    float64 
 9   Color_intensity                 178 non-null    float64 
 10  Hue                             178 non-null    float64 
 11  OD280%2FOD315_of_diluted_wines  178 non-null    float64 
 12  Proline               

# Transform categorical variable to numeric

In [10]:
le=LabelEncoder()

iris_y=le.fit_transform(iris_label)
wine_y=le.fit_transform(wine_label)

# Min-max normalise

In [11]:
scaler = MinMaxScaler()

iris_x_scaled=scaler.fit_transform(iris_x)
wine_x_scaled=scaler.fit_transform(wine_x)


# DataSets

In [12]:
Dataset = ["Iris","Wine"]

# KMeans

In [13]:
def algorithm (x, y, parameters):
    start_time = time.time()
    kmeans =KMeans(n_clusters=parameters[0], max_iter=parameters[1], n_init=parameters[2])
    y_predict = kmeans.fit_predict(x)
    f1score = f1_score(y, y_predict, average = 'weighted')
    ars = adjusted_rand_score(y, y_predict)
    sscore = silhouette_score(x, y_predict, metric='euclidean')
    execution_time = time.time() - start_time
    return y_predict, f1score, ars, sscore, execution_time

In [14]:
# def algorithm(x, y, algo):
#     start_time = time.time()
#     y_predict = algo.fit_predict(x)
#     f1score = f1_score(y, y_predict, average = 'weighted')
#     ars = adjusted_rand_score(y, y_predict)
#     sscore = silhouette_score(x, y_predict, metric='euclidean')
#     execution_time = time.time() - start_time
#     return y_predict, f1score, ars, sscore, execution_time

In [15]:
n_clusters=[2,3,4,5,6,7,8,9,10,20,21,22,23,24,25]
max_iter=[200,300,400]
n_init=[5,10,15]
    
kmean_parameters = pd.DataFrame({"n_clusters":[] , 
                                 "max_iter":[] , 
                                 "n_init":[]}).astype(int)

for i in n_clusters:
        for  ite in max_iter:
            for n in n_init:
                kmean_parameters = kmean_parameters.append({"n_clusters" : i, 
                                                            "max_iter" : ite, 
                                                            "n_init" : n},ignore_index=True)
                
                
kmean_parameters

,n_clusters,max_iter,n_init
0,2,200,5
1,2,200,10
2,2,200,15
3,2,300,5
4,2,300,10
...,...,...,...
130,25,300,10
131,25,300,15
132,25,400,5
133,25,400,10


In [16]:
# kmeans =KMeans(n_clusters=kmean_parameters.iloc[:z,:1].values, max_iter=kmean_parameters.iloc[:z,:2].values, n_init=kmean_parameters.iloc[:z,:3].values)

In [17]:
final_iris_kmeans = Parallel(n_jobs=-1)(delayed(algorithm)(iris_x_scaled, iris_y, kmean_parameters.iloc[i]) for i in range(0, len(kmean_parameters)))
final_wine_kmeans = Parallel(n_jobs=-1)(delayed(algorithm)(wine_x_scaled, wine_y, kmean_parameters.iloc[i]) for i in range(0, len(kmean_parameters)))

In [18]:
# final_iris_kmeans = Parallel(n_jobs=-1)(delayed(algorithm)(iris_x_scaled, iris_y, kmeans) for z in range(0, len(kmean_parameters)))
# final_wine_kmeans = Parallel(n_jobs=-1)(delayed(algorithm)(wine_x_scaled, wine_y, kmeans) for z in range(0, len(kmean_parameters)))

# final_iris_kmeans

In [19]:
final_kmeans_df=pd.DataFrame({
    "Dataset":[],
    "[n_clusters,max_iter,n_init]":[],
    "f1 score":[],
    "Adjusted Random Score":[],
    "Silhouette Score":[],
    "Execution Time":[],
})

In [20]:
for i in range(0, len(kmean_parameters)):
    final_kmeans_df =final_kmeans_df.append({
        "Dataset":Dataset[0],
        "[n_clusters,max_iter,n_init]":kmean_parameters.iloc[i].to_list(),
        "f1 score":final_iris_kmeans[i][1],
        "Adjusted Random Score":final_iris_kmeans[i][2],
        "Silhouette Score":final_iris_kmeans[i][3],
        "Execution Time":final_iris_kmeans[i][4],
        },ignore_index=True)
    
for i in range(0, len(kmean_parameters)):
    final_kmeans_df =final_kmeans_df.append({
        "Dataset":Dataset[0],
        "[n_clusters,max_iter,n_init]":kmean_parameters.iloc[i].to_list(),
        "f1 score":final_wine_kmeans[i][1],
        "Adjusted Random Score":final_wine_kmeans[i][2],
        "Silhouette Score":final_wine_kmeans[i][3],
        "Execution Time":final_wine_kmeans[i][4],
        "Dataset":Dataset[1]
        },ignore_index=True)
    
final_kmeans_df

,Dataset,"[n_clusters,max_iter,n_init]",f1 score,Adjusted Random Score,Silhouette Score,Execution Time
0,Iris,"[2, 200, 5]",0.000000,0.568116,0.629468,0.062486
1,Iris,"[2, 200, 10]",0.000000,0.568116,0.629468,0.031242
2,Iris,"[2, 200, 15]",0.555556,0.568116,0.629468,0.078107
3,Iris,"[2, 300, 5]",0.000000,0.568116,0.629468,0.015621
4,Iris,"[2, 300, 10]",0.000000,0.568116,0.629468,0.031241
...,...,...,...,...,...,...
265,Wine,"[25, 300, 10]",0.000000,0.155908,0.128247,0.140594
266,Wine,"[25, 300, 15]",0.000000,0.200542,0.144015,0.187458
267,Wine,"[25, 400, 5]",0.142227,0.161754,0.136670,0.078106
268,Wine,"[25, 400, 10]",0.281910,0.166412,0.140221,0.124971


In [21]:
final_kmeans_df.to_csv('K-Means_Data.csv')

# Agglomerative Clustering

In [22]:
def algorithm (x, y, parameters):
    start_time = time.time()
    agglomerative =AgglomerativeClustering(n_clusters=parameters[0], linkage=parameters[1])
    y_predict = agglomerative.fit_predict(x)
    f1score = f1_score(y, y_predict, average = 'weighted')
    ars = adjusted_rand_score(y, y_predict)
    sscore = silhouette_score(x, y_predict, metric='euclidean')
    execution_time = time.time() - start_time
    return y_predict, f1score, ars, sscore, execution_time

In [23]:
n_clusters =[2,3,4,5,6,7,8,9,10,11,12,13,14,15]
linkage = ["ward", "complete", "average", "single"]

agglomerative_parameters = pd.DataFrame({
    "n_clusters":[] ,
    "linkage":[]}).astype(int)
  
for i in n_clusters:
        for  n in linkage:
                agglomerative_parameters = agglomerative_parameters.append({
                    "n_clusters" : i, 
                    "linkage" : n },
                    ignore_index=True)
                
agglomerative_parameters

,n_clusters,linkage
0,2,ward
1,2,complete
2,2,average
3,2,single
4,3,ward
5,3,complete
6,3,average
7,3,single
8,4,ward
9,4,complete


In [24]:
final_iris_aggromilative = Parallel(n_jobs=-1)(delayed(algorithm)(iris_x_scaled, iris_y, agglomerative_parameters.iloc[i]) for i in range(0, len(agglomerative_parameters)))
final_wine_aggromilative = Parallel(n_jobs=-1)(delayed(algorithm)(wine_x_scaled, wine_y, agglomerative_parameters.iloc[i]) for i in range(0, len(agglomerative_parameters)))

In [25]:
final_aggromilative_df=pd.DataFrame({
    "Dataset":[],
    "[n_clusters,linkage]":[],
    "f1 score":[],
    "Adjusted Random Score":[],
    "Silhouette Score":[],
    "Execution Time":[],
})

In [26]:
for i in range(0, len(agglomerative_parameters)):
    final_aggromilative_df =final_aggromilative_df.append({
        "Dataset":Dataset[0],
        "[n_clusters,linkage]":agglomerative_parameters.iloc[i].to_list(),
        "f1 score":final_iris_aggromilative[i][1],
        "Adjusted Random Score":final_iris_aggromilative[i][2],
        "Silhouette Score":final_iris_aggromilative[i][3],
        "Execution Time":final_iris_aggromilative[i][4],
        },ignore_index=True)
    
for i in range(0, len(agglomerative_parameters)):
    final_aggromilative_df =final_aggromilative_df.append({
        "Dataset":Dataset[1],
        "[n_clusters,linkage]":agglomerative_parameters.iloc[i].to_list(),
        "f1 score":final_wine_aggromilative[i][1],
        "Adjusted Random Score":final_wine_aggromilative[i][2],
        "Silhouette Score":final_wine_aggromilative[i][3],
        "Execution Time":final_wine_aggromilative[i][4],
        },ignore_index=True)
    
final_aggromilative_df

,Dataset,"[n_clusters,linkage]",f1 score,Adjusted Random Score,Silhouette Score,Execution Time
0,Iris,"[2, ward]",0.000000,0.568116,0.629468,0.127101
1,Iris,"[2, complete]",0.208740,0.223420,0.303913,0.110476
2,Iris,"[2, average]",0.000000,0.568116,0.629468,0.109480
3,Iris,"[2, single]",0.555556,0.568116,0.629468,0.108481
4,Iris,"[3, ward]",0.265060,0.719584,0.504349,0.015628
...,...,...,...,...,...,...
107,Wine,"[14, single]",0.175392,-0.012347,-0.120288,0.015623
108,Wine,"[15, ward]",0.280516,0.288701,0.142652,0.015623
109,Wine,"[15, complete]",0.543024,0.515401,0.186428,0.015623
110,Wine,"[15, average]",0.343167,0.684635,0.178581,0.000000


In [27]:
final_aggromilative_df.to_csv('Aggromilative_Data.csv')

# DBScan Clustering

In [28]:
def algorithm (x, y, parameters):
    start_time = time.time()
    dbscan= DBSCAN(eps = parameters[0], min_samples = parameters[1])
    y_predict = dbscan.fit_predict(x)
    f1score = f1_score(y, y_predict, average = "weighted")
    ars = adjusted_rand_score(y, y_predict)
    dbscan_n_clusters = len(set(y_predict)) - (1 if -1 in y_predict else 0)
    if dbscan_n_clusters>=2:
        sscore = silhouette_score(x, y_predict, metric="euclidean")
    else:
        sscore = print("None")
    execution_time = time.time() - start_time
    return y_predict, f1score,ars,sscore, execution_time

In [29]:
eps = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
min_samples = [2,3,4,5,6,7,8]

dbscan_parameters = pd.DataFrame({
    "eps":[],
    "min_samples" :[]
}).astype(int)

for i in eps:
    for n in min_samples:
        dbscan_parameters=dbscan_parameters.append({
            "eps" : i,
            "min_samples": n
        },ignore_index=True)
        
dbscan_parameters

,eps,min_samples
0,0.1,2.0
1,0.1,3.0
2,0.1,4.0
3,0.1,5.0
4,0.1,6.0
...,...,...
58,0.9,4.0
59,0.9,5.0
60,0.9,6.0
61,0.9,7.0


In [30]:
final_iris_dbscan = Parallel(n_jobs=-1)(delayed(algorithm)(iris_x_scaled, iris_y, dbscan_parameters.iloc[i]) for i in range(0, len(dbscan_parameters)))
final_wine_dbscan = Parallel(n_jobs=-1)(delayed(algorithm)(wine_x_scaled, wine_y, dbscan_parameters.iloc[i]) for i in range(0, len(dbscan_parameters)))

In [31]:
final_dbscan_df=pd.DataFrame({
    "Dataset":[],
    "[eps,min_samples]":[],
    "f1 score":[],
    "Adjusted Random Score":[],
    "Silhouette Score":[],
    "Execution Time":[],
})

In [32]:
for i in range(0, len(dbscan_parameters)):
    final_dbscan_df =final_dbscan_df.append({
        "Dataset":Dataset[0],
        "[eps,min_samples]":dbscan_parameters.iloc[i].to_list(),
        "f1 score":final_iris_dbscan[i][1],
        "Adjusted Random Score":final_iris_dbscan[i][2],
        "Silhouette Score":final_iris_dbscan[i][3],
        "Execution Time":final_iris_dbscan[i][4],
        },ignore_index=True)

for i in range(0, len(dbscan_parameters)):
    final_dbscan_df =final_dbscan_df.append({
        "Dataset":Dataset[1],
        "[eps,min_samples]":dbscan_parameters.iloc[i].to_list(),
        "f1 score":final_wine_dbscan[i][1],
        "Adjusted Random Score":final_wine_dbscan[i][2],
        "Silhouette Score":final_wine_dbscan[i][3],
        "Execution Time":final_wine_dbscan[i][4],
        },ignore_index=True)
    
    

In [33]:
final_dbscan_df

,Dataset,"[eps,min_samples]",f1 score,Adjusted Random Score,Silhouette Score,Execution Time
0,Iris,"[0.1, 2.0]",0.505427,0.426260,0.230234,0.031913
1,Iris,"[0.1, 3.0]",0.505427,0.428689,0.202901,0.035916
2,Iris,"[0.1, 4.0]",0.473502,0.393767,0.134221,0.029920
3,Iris,"[0.1, 5.0]",0.406043,0.441407,0.057464,0.028921
4,Iris,"[0.1, 6.0]",0.461289,0.366987,0.057155,0.008979
...,...,...,...,...,...,...
121,Wine,"[0.9, 4.0]",0.165031,0.000000,None,0.015621
122,Wine,"[0.9, 5.0]",0.165031,0.000000,None,0.000000
123,Wine,"[0.9, 6.0]",0.165031,0.000000,None,0.000000
124,Wine,"[0.9, 7.0]",0.165031,0.000000,None,0.000000


In [34]:
final_dbscan_df.to_csv('DBScan_Data.csv')

In [35]:
# n_clusters = len(set(y_dbscan)) - (1 if -1 in y_dbscan else 0)
# n_noise = list(y_dbscan).count(-1)

# print('Estimated number of clusters: %d' % n_clusters)
# print('Estimated number of noise points: %d' % n_noise)

Unlike k-means, DBSCAN will figure out the number of clusters. 
DBSCAN works by determining whether the minimum number of points are close enough to 
one another to be considered part of a single cluster. DBSCAN is very sensitive to scale since 
epsilon is a fixed value for the maximum distance between two points.

# Optics Clustering

# Gaussian mixtures Clustering

# Affinity propagation

In [36]:
def algorithm (x, y, parameters):
    start_time = time.time()
    ap =AffinityPropagation(preference=parameters[0], damping=parameters[1])
    y_predict = ap.fit_predict(x)
    f1score = f1_score(y, y_predict, average = 'weighted')
    ars = adjusted_rand_score(y, y_predict)
    op_n_clusters = len(set(y_predict)) - (1 if -1 in y_predict else 0)
    if op_n_clusters>=2:
        sscore = silhouette_score(x, y_predict, metric="euclidean")
    else:
        sscore = print("None")
    execution_time = time.time() - start_time
    return y_predict, f1score, sscore, ars,  execution_time

In [37]:
preference = [-1,-3,-5,-7,-9,-11,-13]
damping = [0.5,0.7,0.9]

ap_parameters = pd.DataFrame({
    "preference":[],
    "damping" :[]
}).astype(int)

for i in preference:
    for n in damping:
        ap_parameters=ap_parameters.append({
            "preference":i,
            "damping" :n
        },ignore_index=True)
        
ap_parameters

,preference,damping
0,-1.0,0.5
1,-1.0,0.7
2,-1.0,0.9
3,-3.0,0.5
4,-3.0,0.7
5,-3.0,0.9
6,-5.0,0.5
7,-5.0,0.7
8,-5.0,0.9
9,-7.0,0.5


In [38]:
final_iris_ap = Parallel(n_jobs=-1)(delayed(algorithm)(iris_x_scaled, iris_y, ap_parameters.iloc[i]) for i in range(0, len(ap_parameters)))
final_wine_ap = Parallel(n_jobs=-1)(delayed(algorithm)(wine_x_scaled, wine_y, ap_parameters.iloc[i]) for i in range(0, len(ap_parameters)))

In [39]:
final_ap_df=pd.DataFrame({
    "Dataset":[],
    "[preference, damping]":[],
    "f1 score":[],
    "Adjusted Random Score":[],
    "Silhouette Score":[],
    "Execution Time":[],
})

In [40]:
for i in range(0, len(ap_parameters)):
    final_ap_df =final_ap_df.append({
        "Dataset":Dataset[0],
        "[preference, damping]":ap_parameters.iloc[i].to_list(),
        "f1 score":final_iris_ap[i][1],
        "Adjusted Random Score":final_iris_ap[i][2],
        "Silhouette Score":final_iris_ap[i][3],
        "Execution Time":final_iris_ap[i][4],
        },ignore_index=True)
    
for i in range(0, len(ap_parameters)):
    final_ap_df =final_ap_df.append({
        "Dataset":Dataset[1],
        "[preference, damping]":ap_parameters.iloc[i].to_list(),
        "f1 score":final_wine_ap[i][1],
        "Adjusted Random Score":final_wine_ap[i][2],
        "Silhouette Score":final_wine_ap[i][3],
        "Execution Time":final_wine_ap[i][4],
        },ignore_index=True)
    
    
final_ap_df 

,Dataset,"[preference, damping]",f1 score,Adjusted Random Score,Silhouette Score,Execution Time
0,Iris,"[-1.0, 0.5]",0.297129,0.315135,0.450074,0.115990
1,Iris,"[-1.0, 0.7]",0.780876,0.426983,0.664774,0.071808
2,Iris,"[-1.0, 0.9]",0.559006,0.435747,0.662306,0.121975
3,Iris,"[-3.0, 0.5]",0.898281,0.506163,0.744526,0.088065
4,Iris,"[-3.0, 0.7]",0.906329,0.489912,0.756532,0.043203
5,Iris,"[-3.0, 0.9]",0.906329,0.489912,0.756532,0.062486
6,Iris,"[-5.0, 0.5]",0.899183,0.480108,0.742975,0.062488
7,Iris,"[-5.0, 0.7]",0.893333,0.482575,0.727543,0.031245
8,Iris,"[-5.0, 0.9]",0.906329,0.489912,0.756532,0.062485
9,Iris,"[-7.0, 0.5]",0.000000,None,0.000000,0.296803


In [41]:
final_ap_df.to_csv('Affinity_Propagation_Data.csv')

# Mean-shift 

In [42]:
def algorithm (x, y, parameters):
    start_time = time.time()
    meanshift =MeanShift(bandwidth =parameters[0])
    y_predict = meanshift.fit_predict(x)
    f1score = f1_score(y, y_predict, average = 'weighted')
    ars = adjusted_rand_score(y, y_predict)
    meanshift_n_clusters = len(set(y_predict))
    if 10>=meanshift_n_clusters>=2 :
        sscore = silhouette_score(x, y_predict, metric="euclidean")
    else:
        sscore = print ("None")
    execution_time = time.time() - start_time
    return y_predict, f1score, ars, sscore, execution_time

In [43]:
bandwidth = [0.2,0.3,0.4,0.5,0.6,0.7,0.8]

meanshift_parameters = pd.DataFrame({
    "bandwidth":[],
}).astype(int)

for i in bandwidth:
        meanshift_parameters=meanshift_parameters.append({
            "bandwidth":i,
        },ignore_index=True)
        
meanshift_parameters

,bandwidth
0,0.2
1,0.3
2,0.4
3,0.5
4,0.6
5,0.7
6,0.8


In [44]:
final_iris_meanshift = Parallel(n_jobs=-1)(delayed(algorithm)(iris_x_scaled, iris_y, meanshift_parameters.iloc[i]) for i in range(0, len(meanshift_parameters)))
final_wine_meanshift = Parallel(n_jobs=-1)(delayed(algorithm)(wine_x_scaled, wine_y, meanshift_parameters.iloc[i]) for i in range(0, len(meanshift_parameters)))

In [45]:
final_meanshift_df=pd.DataFrame({
    "Dataset":[],
    "[bandwidth]":[],
    "f1 score":[],
    "Adjusted Random Score":[],
    "Silhouette Score":[],
    "Execution Time":[],
})

In [46]:
for i in range(0, len(meanshift_parameters)):
    final_meanshift_df =final_meanshift_df.append({
        "Dataset":Dataset[0],
        "[bandwidth]":meanshift_parameters.iloc[i].to_list(),
        "f1 score":final_iris_meanshift[i][1],
        "Adjusted Random Score":final_iris_meanshift[i][2],
        "Silhouette Score":final_iris_meanshift[i][3],
        "Execution Time":final_iris_meanshift[i][4],
        },ignore_index=True)
    
for i in range(0, len(meanshift_parameters)):
    final_meanshift_df =final_meanshift_df.append({
        "Dataset":Dataset[1],
        "[bandwidth]":meanshift_parameters.iloc[i].to_list(),
        "f1 score":final_wine_meanshift[i][1],
        "Adjusted Random Score":final_wine_meanshift[i][2],
        "Silhouette Score":final_wine_meanshift[i][3],
        "Execution Time":final_wine_meanshift[i][4],
        },ignore_index=True)
    
    
final_meanshift_df 

,Dataset,[bandwidth],f1 score,Adjusted Random Score,Silhouette Score,Execution Time
0,Iris,[0.2],0.494468,0.602107,0.347057,1.537885
1,Iris,[0.3],0.183575,0.593487,0.476486,1.795197
2,Iris,[0.4],0.000000,0.568116,0.629468,1.349391
3,Iris,[0.5],0.000000,0.568116,0.629468,1.248660
4,Iris,[0.6],0.166667,0.000000,None,0.740020
5,Iris,[0.7],0.166667,0.000000,None,0.928514
6,Iris,[0.8],0.166667,0.000000,None,0.544543
7,Wine,[0.2],0.011049,0.000000,None,0.394918
8,Wine,[0.3],0.079155,0.022753,None,0.408908
9,Wine,[0.4],0.550369,0.266724,None,0.860698


In [47]:
final_meanshift_df.to_csv('Meanshift_Data.csv')

# Spectral Clustering

In [48]:
def algorithm (x, y, parameters):
    start_time = time.time()
    spectral =SpectralClustering(n_clusters =parameters[0], affinity=parameters[1], eigen_solver=parameters[2])
    y_predict = spectral.fit_predict(x)
    f1score = f1_score(y, y_predict, average = 'weighted')
    ars = adjusted_rand_score(y, y_predict)
    sscore = silhouette_score(x, y_predict, metric="euclidean")
    execution_time = time.time() - start_time
    return y_predict, f1score, ars, sscore, execution_time      

In [49]:
n_clusters=[2,3,4,5,6,7,8]
affinity=["nearest_neighbors","rbf","precomputed","precomputed_nearest_neighbors"]
eigen_solver=["arpack", "lobpcg", "amg"]

    
spectral_parameters = pd.DataFrame({"n_clusters":[] , 
                                 "affinity":[] , 
                                 "eigen_solver":[]}).astype(int)

for i in n_clusters:
        for  af in affinity:
            for n in eigen_solver:
                spectral_parameters = spectral_parameters.append({"n_clusters" : i, 
                                                            "affinity" : af, 
                                                            "eigen_solver" : n},ignore_index=True)
                
                
spectral_parameters

,n_clusters,affinity,eigen_solver
0,2,nearest_neighbors,arpack
1,2,nearest_neighbors,lobpcg
2,2,nearest_neighbors,amg
3,2,rbf,arpack
4,2,rbf,lobpcg
...,...,...,...
79,8,precomputed,lobpcg
80,8,precomputed,amg
81,8,precomputed_nearest_neighbors,arpack
82,8,precomputed_nearest_neighbors,lobpcg


In [50]:
final_iris_spectral = Parallel(n_jobs=-1)(delayed(algorithm)(iris_x_scaled, iris_y, spectral_parameters.iloc[i]) for i in range(0, len(ap_parameters)))
# final_wine_ap = Parallel(n_jobs=-1)(delayed(algorithm)(wine_x_scaled, wine_y, ap_parameters.iloc[i]) for i in range(0, len(ap_parameters)))

ValueError: The eigen_solver was set to 'amg', but pyamg is not available.

In [52]:
python setup.py install

SyntaxError: invalid syntax (Temp/ipykernel_8444/1000827859.py, line 1)

# Ward hierarchical

In [ ]:
from sklearn import metrics
labels_true = [0, 0, 0, 1, 1, 1]
labels_pred = [0, 0, 0, 1, 1, 2]
metrics.rand_score(labels_true, labels_pred)

In [ ]:
ap =MeanShift(bandwidth=0.75)
y_predict = ap.fit_predict(wine_x_scaled)

y_predict